In [9]:
import torch
import torch.nn as nn
import random

In [10]:
def make_noise(batch, latent_dim, n_noise, device):
    if n_noise == 1:
        return torch.randn(batch, latent_dim, device=device)

    noises = torch.randn(n_noise, batch, latent_dim, device=device).unbind(0)

    return noises


def mixing_noise(batch, latent_dim, prob, device):
    """有prob的概率产生两个噪音张量元组，1-prob的概率产生单个噪音张量"""
    if prob > 0 and random.random() < prob:
        return make_noise(batch, latent_dim, 2, device)

    else:
        return [make_noise(batch, latent_dim, 1, device)]

In [7]:
type(make_noise(4, 12, 2, 'cpu'))

tuple

In [8]:
type(make_noise(4, 12, 1, 'cpu'))

torch.Tensor

In [18]:
mixing_noise(4, 12, 0.3, 'cpu')

[tensor([[ 0.6824,  0.5227, -0.8227,  0.1259, -0.2245, -0.3000, -0.9665, -0.7178,
           0.7661, -0.9234,  1.9271, -1.6021],
         [-0.4725,  0.1024, -1.5735,  0.1093,  0.5130, -0.0407,  0.4853, -0.0145,
           0.4717,  0.9101,  0.2909,  0.1696],
         [ 0.7514,  0.4667, -1.4836, -0.7291, -1.0503, -1.4260,  0.6439, -0.5074,
           0.4496, -0.2059, -1.4044, -2.2906],
         [ 1.9708, -0.3942,  0.1509,  0.9212,  0.3539,  0.5483, -0.4377, -0.3167,
           0.0702,  0.1179,  0.3041, -1.8386]])]